In [2]:
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

In [3]:
class LinearRegressionWrapper():
    def __init__(self, model_params={'fit_intercept': False}):

        self.model_name = "linear_regression"
        self.search_type = 'grid'
        self.param_grid = {'fit_intercept': [True, False]}
        if model_params is None:
            self.ModelClass = LinearRegression()
        else:
            self.ModelClass = LinearRegression(**model_params)

In [4]:
train_df = pd.read_csv('train_data.csv')
train_df.set_index("date", inplace=True)

train_df.tail()

,SPY,RPI,S&P: indust,HOUSTW,EXJPUSx,WPSID62
date,,,,,,
2003-08-31,0.020416,0.001636,0.001452,6.063785,-0.000282,-0.008375
2003-09-30,-0.010803,0.001476,0.033748,6.113682,-0.033091,0.027151
2003-10-31,0.052144,0.005737,0.011946,6.317165,-0.002906,-0.002173
2003-11-30,0.010862,0.005737,0.011946,6.317165,-0.002906,-0.002173
2003-12-31,0.048997,0.000350,0.031326,6.253829,-0.013278,0.032111


In [5]:
train_df.shape

(48, 6)

In [16]:
target = "SPY"
k = 5
scoring = "neg_mean_squared_error"
model_wrapper = LinearRegressionWrapper(model_params={'fit_intercept': False})

y = train_df[target]
X = train_df.drop(columns=[target])
forecast_model = model_wrapper.ModelClass

In [21]:
sfs_cv = SequentialFeatureSelector(forecast_model, scoring=scoring)
sfs_cv.fit(X, y)

selected_features = sfs_cv.get_support(indices=True)
print(f"Selected features: {X.columns[selected_features]}")

Selected features: Index(['RPI', 'S&P: indust'], dtype='object')


In [23]:
tscv = TimeSeriesSplit(n_splits=k)
sfs_tscv = SequentialFeatureSelector(forecast_model, cv=tscv, scoring=scoring)
fit = sfs_tscv.fit(X, y)

selected_features = sfs_tscv.get_support(indices=True)